# Baseline
- predikovat biologickou aktivitu pro nejaky baseline asi randomforest

In [16]:
import string
import pandas as pd
import csv

from sklearn import metrics
import matplotlib.pyplot as plt
import numpy as np

from sklearn.model_selection import KFold

from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem import AllChem
from rdkit.Chem.Draw import IPythonConsole

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

In [10]:
df = pd.read_csv('../data/VEGF2_ChEMBL28-10980_pic50_noDuplicate_canSmiles_sln_deepSmiles_selfies_inchi.csv',sep=',')

linearNotation_names = ['canonical_smiles','sln','deep_smiles','selfies','inchi']
for notName in linearNotation_names:
    df[notName+'_length'] = df[notName].str.len()
print(df.shape)
# canonical_smiles_length
df = df.loc[(df['canonical_smiles_length'] >30) & (df['canonical_smiles_length']<80)]
print(df.shape)

(7159, 11)
(7044, 11)


In [11]:
# # puvodni chemicke linearni zapisy
# df_full = pd.read_csv('../data/VEGF2_ChEMBL28-10980_pic50_noDuplicate_canSmiles_sln_deepSmiles_selfies_inchi.csv',sep=',')
# # df_full = pd.read_csv('../data/VEGF2_ChEMBL28-10980_pic50_oneHotEncoding_all_chem_linear_notaion.csv',sep=',')
# df = df_full[['canonical_smiles','value','sln','deep_smiles','selfies','inchi']]
df.head(2)

,canonical_smiles,value,sln,deep_smiles,selfies,inchi,canonical_smiles_length,sln_length,deep_smiles_length,selfies_length,inchi_length
0,Br.COc1cc(OC)cc(N(CCP(=O)(O)O)c2ccc3ncc(-c4cnn...,7.32,BrH,Br.COcccOC))ccNCCP=O)O)O))))ccccncc-ccnnC)c5))...,[Br].[C][O][C][=C][C][Branch1_1][Ring1][O][C][...,InChI=1S/C22H24N5O5P.BrH/c1-26-14-15(12-24-26)...,60,3,69,292,168
1,Brc1ccc(-c2nc3ccc(Nc4ccnc5ccccc45)cc3[nH]2)cc1,6.75,BrC[5]=CHCH=C(C[4]=NC[3]=CHCH=C(NHC[1]=CHCH=NC...,Brcccc-cnccccNcccncccccc%106)))))))))))cc6[nH]...,[Br][C][=C][C][=C][Branch2_1][Ring1][S][C][=N]...,InChI=1S/C22H15BrN4/c23-15-7-5-14(6-8-15)22-26...,46,84,59,226,133


## Create features
- moje features jsou morganovy fingerprinty (z bakalarky jsem to vzala)

In [12]:
RANDOM_STATE = 42
#prevedu smiles na mol 
def molFromSmiles(smiles):
    return (Chem.MolFromSmiles(smiles))

# ulozim si mol do sloupecku v dataframu
df['mol'] = df['canonical_smiles'].apply(molFromSmiles)
np_mol = np.asarray(df['mol'])
    
# vypoctu si pro kazdy smiles fingerprint
fps = [AllChem.GetMorganFingerprintAsBitVect(m, 2, nBits=1024) for m in np_mol]
# convert the RDKit explicit vectors into numpy arrays
np_fps = np.asarray(fps)

# X - features array
# y - target variable/prediction
X=np_fps  # Features
y=df['value']  # Labels

# kvuli ladeni hyperparametru u prediction model
# Split dataset into train, validate and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=RANDOM_STATE)  # 80% training and 20% test
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=RANDOM_STATE)

In [13]:
y

0       7.320000
1       6.750000
2       6.420000
3       7.850000
5       7.453333
          ...   
7154    6.440000
7155    7.700000
7156    5.470000
7157    5.525000
7158    6.030000
Name: value, Length: 7044, dtype: float64

## Random hyperparameter grid

In [14]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


In [18]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
kfold = KFold(n_splits=10, shuffle=True, random_state=RANDOM_STATE)
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = kfold, verbose=2, random_state=RANDOM_STATE, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train)

Fitting 10 folds for each of 100 candidates, totalling 1000 fits


RandomizedSearchCV(cv=KFold(n_splits=10, random_state=42, shuffle=True),
                   estimator=RandomForestRegressor(), n_iter=100, n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=42, verbose=2)

In [20]:
# parametry pro nejlepsi strom
best_std = rf_random.cv_results_['std_test_score'][rf_random.best_index_]
print(best_std)
best_params  = rf_random.best_params_
print(best_params)

0.03115296127121132
{'n_estimators': 1400, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 80, 'bootstrap': False}


qstat{'n_estimators': 1400,
 'min_samples_split': 5,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': 80,
 'bootstrap': False}

## Evaluate jednodychy random forest

In [21]:
base_model = RandomForestRegressor(n_estimators=10,random_state = RANDOM_STATE)
base_model.fit(X_train, y_train)
y_pred = base_model.predict(X_test)

# print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
# base_accuracy = evaluate(base_model, test_features, test_labels)

Mean Squared Error: 0.5192958044365671
Root Mean Squared Error: 0.7206218179021275


## Evaluate Random Search

In [22]:
# vypdrnu se na grid search i random serach trva pomerne dlooooouho
best_random = rf_random.best_estimator_
y_pred = best_random .predict(X_test)
# random_accuracy = evaluate(best_random, test_features, test_labels)

# print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Squared Error: 0.4392467727293858
Root Mean Squared Error: 0.6627569484580194


---------------------------------------------------------------------------------------------------------------------------
# Vynechat
- grid search je hodne pomaly, zvolila jsem radsi jen randomized search

### Grid Search with Cross Validation

In [ ]:
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}
# Create a based model
rf = RandomForestRegressor()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, cv = 3, n_jobs = -1, verbose = 2)

In [ ]:
# Fit the grid search to the data
grid_search.fit(X_train, y_train)
grid_search.best_params_

In [ ]:
best_grid = grid_search.best_estimator_

In [ ]:
y_pred = best_grid.predict(X_val)
# random_accuracy = evaluate(best_random, test_features, test_labels)

# print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_val, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_val, y_pred)))

### Vybran random search
- pro hledani hyperparametry

In [ ]:
best_random = rf_random.best_estimator_
y_pred = best_random .predict(X_test)
# random_accuracy = evaluate(best_random, test_features, test_labels)

# print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_tes, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))